# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/neSFl/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/dsECZ/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298498410894                   -0.85    5.1         
  2   -8.300196054035       -2.77       -1.25    1.1    114ms
  3   -8.300430742130       -3.63       -1.88    1.4    121ms
  4   -8.300460695425       -4.52       -2.75    3.8    161ms
  5   -8.300464032729       -5.48       -3.07    2.9    209ms
  6   -8.300464317255       -6.55       -3.24    8.6    223ms
  7   -8.300464480367       -6.79       -3.39    1.8    130ms
  8   -8.300464565114       -7.07       -3.53    1.2    120ms
  9   -8.300464626312       -7.21       -3.76    3.1    168ms
 10   -8.300464632181       -8.23       -3.85    1.0    155ms
 11   -8.300464642229       -8.00       -4.11    2.6    158ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67480755587                   -0.70    6.4         
  2   -16.67879693906       -2.40       -1.14    1.0    300ms
  3   -16.67922961801       -3.36       -1.87    2.1    320ms
  4   -16.67927646579       -4.33       -2.77    3.1    350ms
  5   -16.67928594977       -5.02       -3.20    5.6    477ms
  6   -16.67928619651       -6.61       -3.49    2.2    310ms
  7   -16.67928621828       -7.66       -3.98    2.1    317ms
  8   -16.67928622143       -8.50       -4.59    2.6    349ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32524649378                   -0.56    6.9         
  2   -33.33256914443       -2.14       -1.00    1.0    1.03s
  3   -33.33410900210       -2.81       -1.73    4.5    1.37s
  4   -33.33428364001       -3.76       -2.64    2.6    1.19s
  5   -33.33694043753       -2.58       -2.50    6.9    1.70s
  6   -33.33694135514       -6.04       -2.54    7.9    1.47s
  7   -33.33694364454       -5.64       -3.04    1.0    959ms
  8   -33.33694374397       -7.00       -3.56    4.6    1.24s
  9   -33.33694377823       -7.47       -3.99    3.6    1.20s
 10   -33.33694378269       -8.35       -4.45    3.6    1.30s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298489372514                   -0.85    5.1         
  2   -8.300255626010       -2.75       -1.59    1.0   90.2ms
  3   -8.300420034504       -3.78       -2.48    2.1    114ms
  4   -8.300355734125   +   -4.19       -2.23    8.6    231ms
  5   -8.300463992741       -3.97       -3.43    1.0   97.6ms
  6   -8.300464565090       -6.24       -3.74    3.9    203ms
  7   -8.300464633915       -7.16       -4.17    1.1    105ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32583889788                   -0.56    6.6         
  2   -33.28393506118   +   -1.38       -1.25    2.1    933ms
  3   +9.660484247687   +    1.63       -0.25    6.9    2.94s
  4   -33.28757593433        1.63       -1.49    5.8    2.05s
  5   -33.28618120335   +   -2.86       -1.43    2.8    1.31s
  6   -33.31527749347       -1.54       -1.84    2.0    1.09s
  7   -33.13568676210   +   -0.75       -1.41    3.5    1.39s
  8   -33.32665620352       -0.72       -2.03    4.1    1.34s
  9   -33.33648644419       -2.01       -2.56    2.6    1.12s
 10   -33.33656781131       -4.09       -2.54    2.8    1.40s
 11   -33.33688577389       -3.50       -3.01    1.8    1.06s
 12   -33.33691059631       -4.61      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.